
Modelo de regressão para carros usados

In [30]:
# Importando bibliotecas
import pandas as pd
import torch
import numpy as np
import seaborn as sns
from torch import nn, optim
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [48]:
# Carregando a base de dados/ iso é o formato dos caracteres da base de dados
carros = pd.read_csv('/content/autos.csv', encoding = 'ISO-8859-1')

In [49]:
# Observando a base de dados
carros.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [50]:
# Observando numero de registros
carros.shape

(341490, 20)

In [51]:
# Informações
carros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341490 entries, 0 to 341489
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          341490 non-null  object
 1   name                 341490 non-null  object
 2   seller               341490 non-null  object
 3   offerType            341490 non-null  object
 4   price                341490 non-null  int64 
 5   abtest               341490 non-null  object
 6   vehicleType          306721 non-null  object
 7   yearOfRegistration   341490 non-null  int64 
 8   gearbox              322917 non-null  object
 9   powerPS              341490 non-null  int64 
 10  model                322650 non-null  object
 11  kilometer            341490 non-null  int64 
 12  monthOfRegistration  341490 non-null  int64 
 13  fuelType             310804 non-null  object
 14  brand                341490 non-null  object
 15  notRepairedDamage    275281 non-nu

In [52]:
# Verificando a existência de valores nulos
carros.isna().any()

dateCrawled            False
name                   False
seller                 False
offerType              False
price                  False
abtest                 False
vehicleType             True
yearOfRegistration     False
gearbox                 True
powerPS                False
model                   True
kilometer              False
monthOfRegistration    False
fuelType                True
brand                  False
notRepairedDamage       True
dateCreated            False
nrOfPictures           False
postalCode             False
lastSeen               False
dtype: bool

In [53]:
# verificando quantidade de valores nulos
carros.isnull().sum()

dateCrawled                0
name                       0
seller                     0
offerType                  0
price                      0
abtest                     0
vehicleType            34769
yearOfRegistration         0
gearbox                18573
powerPS                    0
model                  18840
kilometer                  0
monthOfRegistration        0
fuelType               30686
brand                      0
notRepairedDamage      66209
dateCreated                0
nrOfPictures               0
postalCode                 0
lastSeen                   0
dtype: int64

In [54]:
# Analisando a variável vehicletype para observar qual o veículo que mais se repete
carros['vehicleType'].value_counts()

limousine     88219
kleinwagen    73508
kombi         62088
bus           27697
cabrio        21158
coupe         17429
suv           13531
andere         3091
Name: vehicleType, dtype: int64

In [55]:
# Analisando a variável gearbox para observar qual o ítem que mais se repete
carros['gearbox'].value_counts()

manuell      252046
automatik     70871
Name: gearbox, dtype: int64

In [56]:
# Analisando a variável model para observar qual o veículo que mais se repete
carros['model'].value_counts()

golf               27666
andere             24301
3er                18906
polo               12056
corsa              11529
                   ...  
kalina                 7
rangerover             6
serie_3                4
serie_1                2
discovery_sport        1
Name: model, Length: 251, dtype: int64

In [57]:
# Analisando a variável fueltype para observar qual o ítem que mais se repete
carros['fuelType'].value_counts()

benzin     205825
diesel      99021
lpg          4894
cng           524
hybrid        252
andere        196
elektro        92
Name: fuelType, dtype: int64

In [58]:
# Analisando a variável notrepairedamage para observar qual o ítem que mais se repete
carros['notRepairedDamage'].value_counts()

nein    241963
ja       33318
Name: notRepairedDamage, dtype: int64

In [59]:
# Fazendo o preenchimento dos valores que mais aparecem no lugar dos nulos
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
carros = carros.fillna(value = valores)

In [60]:
# Observando os valores nulos após o tratamento
carros.isnull().sum()

dateCrawled            0
name                   0
seller                 0
offerType              0
price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dateCreated            0
nrOfPictures           0
postalCode             0
lastSeen               0
dtype: int64

In [61]:
# Apagando algumas colunas irrelevantes
carros = carros.drop('dateCrawled', axis = 1)
carros = carros.drop('dateCreated', axis = 1)
carros = carros.drop('nrOfPictures', axis = 1)
carros = carros.drop('postalCode', axis = 1)
carros = carros.drop('lastSeen', axis = 1)
carros = carros.drop('name', axis = 1) # ja temos a coluna model, portanto são equivalentes
carros = carros.drop('seller', axis = 1)
carros = carros.drop('offerType', axis = 1)

In [62]:
# observando se os atributos foram removidos
carros.head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,limousine,1993,manuell,0,golf,150000,0,benzin,volkswagen,nein
1,18300,test,coupe,2011,manuell,190,golf,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,nein
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein


In [63]:
# Observando se existem preços inconsistentes
# Criando novo dataframe
# Verificando se existem carros que custam menos de 1000 euros
carros1 = carros.loc[carros.price <= 1000]

In [64]:
# Observando
carros1.shape

(80929, 12)

In [65]:
# Pela inconsistência vamos manter os veículos que custam acima de 1000 euros
carros = carros[carros.price > 1000]

In [66]:
# Visualizando
carros.shape

(260561, 12)

In [67]:
# Criando outro dataframe para observar novas inconsistências
carros2 = carros.loc[carros.price > 400000]

In [68]:
carros2.shape

(96, 12)

In [69]:
carros = carros.loc[carros.price < 400000]

In [70]:
# Observando os registros
carros.shape

(260465, 12)

In [71]:
# Definindo as variaveis que vão contribuir para a previsão dos preços
previsores = carros.iloc[:, 1:13].values # da variavel 1 até a 13
valor = carros.iloc[:, 0].values # pegamos apenas a variavel price que é a 0

In [72]:
# Observando a variavel
previsores.shape

(260465, 11)

In [73]:
# Transformando os atributos categóricos em numéricos com o Onehotencoder
# Onde o será abtest / 1 será vehicleType / 3 será gearbox / 5 será model / 8 será fuelType / 9 será brand / 10 será notrepairedamage
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(),
                                                   [0,1,3,5,8,9,10])],
                                  remainder = 'passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

In [74]:
# Observando
previsores.shape

(260465, 316)

In [75]:
# Convertendo de numpy para pytorch
previsores = torch.tensor(previsores, dtype = torch.float)
valor = torch.tensor(valor, dtype = torch.float).view(-1, 1)

In [76]:
# Criando a rede neural 316 atributos de entrada
# entradas(316) + saídas(1) / 2 para iniciar os neurônios na camada oculta da rede neural
# 2 camadas ocultas com 158 neurônios
modelo = nn.Sequential(nn.Linear(316, 158),
                          nn.ReLU(),
                          nn.Linear(158, 158),
                          nn.ReLU(),
                          nn.Linear(158, 1))

In [77]:
# Definindo a função de erro
erro = nn.L1Loss()

In [78]:
# otimizador
otimizador = optim.Adam(modelo.parameters())

In [79]:
# Criando a base de dados no formato do pytorch
dataset = torch.utils.data.TensorDataset(previsores, valor)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=300, shuffle=True)

In [80]:
# Iniciando o treinamento do modelo
# Cuda é um tipo de gpu usada para deep learning
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [81]:
modelo.to(device)

Sequential(
  (0): Linear(in_features=316, out_features=158, bias=True)
  (1): ReLU()
  (2): Linear(in_features=158, out_features=158, bias=True)
  (3): ReLU()
  (4): Linear(in_features=158, out_features=1, bias=True)
)

In [87]:
for epoch in range(100): # Fazendo o treinamento por 100 épocas
  running_loss = 0. # erro
  running_mae = 0.

  for i, data in enumerate(train_loader):
    inputs, labels = data # fazendo processamento através das entradas e dos valores reais

    inputs, labels = inputs.to(device), labels.to(device) # enviando dados para gpu

    otimizador.zero_grad() # zerando os gradientes a cada loop

    outputs = modelo.forward(inputs) # valor da previsão

    mae = F.l1_loss(outputs, labels).item() # calculo do erro

    running_mae += mae # histórico dos valores

    loss = erro(outputs, labels) # calculo do erro para o otimizador
    loss.backward() # atualização dos pesos através do backpropagation

    otimizador.step() # atualizar os pesos

    running_loss += loss.item()

    print('\rÉpoca {:3d} - Loop {:3d} de {:3d}: perda {:06.2f} - MAE {:06.2f}'.format(epoch+1,
                                                                                      i + 1,
                                                                                      len(valor)//300,
                                                                                      loss, mae))
  print('ÉPOCA {:3d} finalizada: perda {:0.5f} - MAE {:0.5f} '.format(epoch+1,
                                                                      running_loss/len(train_loader),
                                                                      running_mae/len(train_loader)))

A saída de streaming foi truncada nas últimas 5000 linhas.
Época  95 - Loop 221 de 868: perda 2467.61 - MAE 2467.61
Época  95 - Loop 222 de 868: perda 3218.42 - MAE 3218.42
Época  95 - Loop 223 de 868: perda 2258.90 - MAE 2258.90
Época  95 - Loop 224 de 868: perda 2378.30 - MAE 2378.30
Época  95 - Loop 225 de 868: perda 2492.31 - MAE 2492.31
Época  95 - Loop 226 de 868: perda 2494.83 - MAE 2494.83
Época  95 - Loop 227 de 868: perda 2538.90 - MAE 2538.90
Época  95 - Loop 228 de 868: perda 2775.84 - MAE 2775.84
Época  95 - Loop 229 de 868: perda 3130.26 - MAE 3130.26
Época  95 - Loop 230 de 868: perda 1842.74 - MAE 1842.74
Época  95 - Loop 231 de 868: perda 2631.70 - MAE 2631.70
Época  95 - Loop 232 de 868: perda 2267.42 - MAE 2267.42
Época  95 - Loop 233 de 868: perda 2714.14 - MAE 2714.14
Época  95 - Loop 234 de 868: perda 2465.88 - MAE 2465.88
Época  95 - Loop 235 de 868: perda 2872.47 - MAE 2872.47
Época  95 - Loop 236 de 868: perda 2264.52 - MAE 2264.52
Época  95 - Loop 237 de 868: 

In [88]:
# Colocando a rede neural em forma de avaliação
modelo.eval()

Sequential(
  (0): Linear(in_features=316, out_features=158, bias=True)
  (1): ReLU()
  (2): Linear(in_features=158, out_features=158, bias=True)
  (3): ReLU()
  (4): Linear(in_features=158, out_features=1, bias=True)
)

In [89]:
# Criando a variavel para as previsões
previsoes = modelo.forward(previsores.to(device))

In [91]:
# Observando o preço previsto para cada carro
previsoes

tensor([[12089.4385],
        [13019.0928],
        [ 2450.3430],
        ...,
        [12882.1016],
        [ 2151.1719],
        [ 3730.0120]], grad_fn=<AddmmBackward0>)

In [92]:
# Observando os preços reais dos carros
valor

tensor([[18300.],
        [ 9800.],
        [ 1500.],
        ...,
        [ 9000.],
        [ 2400.],
        [ 1200.]])

In [93]:
# Tirando a média para saber a divergência dos valores
# Primeiro a média dos preços reais
valor.mean()

tensor(7359.7456)

In [94]:
# Agora dos valores previstos pelo modelo
previsoes.mean()

tensor(7328.3228, grad_fn=<MeanBackward0>)

In [95]:
# Observando quanto o algoritmo está errando nos preços
# preço real - previsoes
7359 - 7328

31